# Introduction

# Read Data

Make sure to have the data locally locally saved as `/data`.

In [2]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import glob

In [3]:
# Load in Artist CSV data
#print(os.listdir("./"))

# get the directory 
main_direc = os.getcwd()
print(main_direc)

artist_csv_loc = os.path.join(main_direc, 'data/artists.csv')
artists = pd.read_csv(artist_csv_loc)
print(artists.shape)
print(artists)

/Users/claykaufmann/Documents/Git-Projects/cs254-final-project
(50, 8)
    id                       name        years  \
0    0          Amedeo Modigliani  1884 - 1920   
1    1         Vasiliy Kandinskiy  1866 - 1944   
2    2               Diego Rivera  1886 - 1957   
3    3               Claude Monet  1840 - 1926   
4    4              Rene Magritte  1898 - 1967   
5    5              Salvador Dali  1904 - 1989   
6    6              Edouard Manet  1832 - 1883   
7    7              Andrei Rublev  1360 - 1430   
8    8           Vincent van Gogh  1853 – 1890   
9    9               Gustav Klimt  1862 - 1918   
10  10           Hieronymus Bosch  1450 - 1516   
11  11           Kazimir Malevich  1879 - 1935   
12  12             Mikhail Vrubel  1856 - 1910   
13  13              Pablo Picasso  1881 - 1973   
14  14          Peter Paul Rubens  1577 - 1640   
15  15      Pierre-Auguste Renoir  1841 - 1919   
16  16             Francisco Goya  1746 - 1828   
17  17                Frida K

# Data Processing

The following approach is a modified version of the approach from [DeepArtists: Identify Artist from Art](https://www.kaggle.com/supratimhaldar/deepartist-identify-artist-from-art).

In [4]:
# Sortting the artists by number of paintings
artists = artists.sort_values(by=['paintings'], ascending=False)
# print(artists)

In [5]:
# Creating a dataframe with the top 10 artists by number of paintings
artists_top = artists.head(10)
artists_top

,id,name,years,genre,nationality,bio,wikipedia,paintings
8,8,Vincent van Gogh,1853 – 1890,Post-Impressionism,Dutch,Vincent Willem van Gogh (Dutch: [ˈvɪnsɛnt ˈʋɪl...,http://en.wikipedia.org/wiki/Vincent_van_Gogh,877
30,30,Edgar Degas,1834 - 1917,Impressionism,French,Edgar Degas (US: or UK: ; born Hilaire-Germai...,http://en.wikipedia.org/wiki/Edgar_Degas,702
13,13,Pablo Picasso,1881 - 1973,Cubism,Spanish,Pablo Ruiz Picasso (; Spanish: [ˈpaβlo piˈkaso...,http://en.wikipedia.org/wiki/Pablo_Picasso,439
15,15,Pierre-Auguste Renoir,1841 - 1919,Impressionism,French,"Pierre-Auguste Renoir, commonly known as Augus...",http://en.wikipedia.org/wiki/Pierre-Auguste_Re...,336
19,19,Albrecht Dürer,1471 - 1528,Northern Renaissance,German,Albrecht Dürer (; German: [ˈʔalbʁɛçt ˈdyːʁɐ]; ...,http://en.wikipedia.org/wiki/Albrecht_Dürer,328
46,46,Paul Gauguin,1848 – 1903,"Symbolism,Post-Impressionism",French,"Eugène Henri Paul Gauguin (UK: , US: ; French:...",http://en.wikipedia.org/wiki/Paul_Gauguin,311
16,16,Francisco Goya,1746 - 1828,Romanticism,Spanish,Francisco José de Goya y Lucientes (; Spanish:...,http://en.wikipedia.org/wiki/Francisco_Goya,291
31,31,Rembrandt,1606 - 1669,Baroque,Dutch,Rembrandt Harmenszoon van Rijn (; Dutch: [ˈrɛm...,http://en.wikipedia.org/wiki/Rembrandt,262
20,20,Alfred Sisley,1839 - 1899,Impressionism,"French,British",Alfred Sisley (; French: [sislɛ]; 30 October 1...,http://en.wikipedia.org/wiki/Alfred_Sisley,259
32,32,Titian,1488 - 1576,"High Renaissance,Mannerism",Italian,Tiziano Vecelli or Tiziano Vecellio (pronounce...,http://en.wikipedia.org/wiki/Titian,255


## Exploring data and creating a DataFrame of all image paths with their associated artists

In [6]:
# Images directories, cut down is just top 10 artists
images_dir = os.path.join(main_direc, 'data/images/images')
cut_down_images_dir = os.path.join(main_direc, 'data/limited_images')

In [7]:
# # Images DataFrame
# artists_top_name = artists_top['name'].str.replace(' ', '_').values

# images_df = pd.DataFrame()
# for name in artists_top_name:
#     #print(glob.glob(images_dir + "/" + name + '/*'))

#     # Method 1:
#     #
#     #images_df = images_df.append(pd.DataFrame(data={'Path': glob.glob('../cs254-final-project/data/images/images/' + name + '/*'), 'Name': name}), ignore_index=True)

#     # Method 2:
#     #
#     images_df = pd.concat([images_df, pd.DataFrame(data={'Path': ["data/images/images/" + name + "/" + os.path.basename(x) for x in glob.glob(images_dir + "/" + name + '/*', )], 'Name': name})], ignore_index=True)

# images_df

In [8]:
# Converting the Images DataFrame to a .csv
# images_df.to_csv(os.path.join(main_direc, 'data/images.csv'), index=True)

## Load In Data

In [7]:
# BEGIN LOGISTIC REGRESSION SECTION
import tensorflow as tf
import PIL.Image
import PIL

# set base parameters for images loader
batch_size = 32
img_height = 299
img_width = 299

# get training dataset (0.25 split)
train_ds = tf.keras.utils.image_dataset_from_directory(
  cut_down_images_dir,
  validation_split=0.25,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

# get testing dataset
vals_ds = tf.keras.utils.image_dataset_from_directory(
  cut_down_images_dir,
  validation_split=0.25,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
vals_ds = vals_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 4060 files belonging to 10 classes.
Using 3045 files for training.
Found 4060 files belonging to 10 classes.
Using 1015 files for validation.


2021-12-07 20:31:34.350906: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## BASELINE MODEL
The following model is a simple convolutional neural network using the keras
Sequential API.

In [9]:
# BUILD MODEL
num_classes = 10

sequential_model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

# compile model
sequential_model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy']
)


In [10]:
# FITTING THE MODEL
sequential_model.fit(
  train_ds,
  validation_data=vals_ds,
  epochs=3
)


Epoch 1/3


2021-12-07 20:31:51.138158: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


KeyboardInterrupt: 

## The following model is Clay Kaufmann's model.
This model takes advantage of transfer learning, using Inception v3.

In [14]:
# import inception
#from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
import tensorflow.keras as K

# set the input for VGG
inp = Input(shape=(img_height,img_width,3))

# load model
base_model = InceptionV3(include_top=False, input_tensor=inp, pooling='max', weights='imagenet')

# set base model to not be trainable
base_model.trainable = False

final_model = tf.keras.Sequential()
final_model.add(base_model)

final_model.add(K.layers.Flatten())
final_model.add(K.layers.BatchNormalization())
final_model.add(Dense(256, activation='relu'))
final_model.add(K.layers.Dropout(0.6))
final_model.add(Dense(num_classes, activation='softmax'))

final_model.summary()

# compile model
final_model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
  metrics=['accuracy']
)

2021-12-07 20:35:14.565421: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2021-12-07 20:35:14.632277: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 2048)              21802784  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
batch_normalization_376 (Bat (None, 2048)              8192      
_________________________________________________________________
dense_3 (Dense)              (None, 256)               524544    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2570      
Total params: 22,338,090
Trainable params: 531,210
Non-trainable params: 21,806,880
____________________________________

In [15]:
# train model
final_model.fit(
  train_ds,
  validation_data=vals_ds,
  epochs=3
)

Epoch 1/3
24/96 [======>.......................] - ETA: 1:31 - loss: 3.9268 - accuracy: 0.2357

Corrupt JPEG data: 10 extraneous bytes before marker 0xd9


96/96 [==============================] - 229s 2s/step - loss: 3.5978 - accuracy: 0.2703 - val_loss: 2.8979 - val_accuracy: 0.3586
Epoch 2/3
96/96 [==============================] - 233s 2s/step - loss: 2.6265 - accuracy: 0.3790 - val_loss: 2.1929 - val_accuracy: 0.4099
Epoch 3/3
96/96 [==============================] - 252s 3s/step - loss: 2.1784 - accuracy: 0.4240 - val_loss: 2.0481 - val_accuracy: 0.4424
